In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import time
import sys
import math
import numpy as np
import random
import joblib
from tqdm import tqdm_notebook

import Config
import Dataloader as DL
import HD_basis as HDB
import HD_encoder as HDE
import HD_classifier as HDC


In [3]:
dl = DL.Dataloader()
nFeatures, nClasses, traindata, trainlabels, testdata, testlabels = dl.getParam()

Loading dataset UCIHAR from UCIHAR
Loading train data... train data of shape (6213, 561) loaded
Loading test data...  test  data of shape (1554, 561) loaded
Data Loaded. Num of features = 561 Num of Classes = 12

In [4]:
param = Config.config
param["nFeatures"] = nFeatures
param["nClasses"] = nClasses

In [5]:
hdb = HDB.HD_basis(HDB.Generator.Vanilla, param)
basis = hdb.getBasis()
bid = hdb.getParam()["id"]
print(bid)

Generated vanilla HD basis of shape... 

(2500, 561)
Encoding time: 0.08776378631591797 
Dumping basis into file: base_1773.pkl 



1773


In [6]:
trainencoded = HDE.encodeData(traindata, basis)
HDE.saveEncoded(trainencoded, trainlabels, bid, "train")

testencoded = HDE.encodeData(testdata, basis)
HDE.saveEncoded(testencoded, testlabels, bid, "test")

Encoding data of shape (6213, 561)


Time spent: 88 sec
Dumping data into encoded_1773_train.pkl 
Encoding data of shape (1554, 561)


Time spent: 22 sec
Dumping data into encoded_1773_test.pkl 


'encoded_1773_test.pkl'

In [9]:
# Load sth 
# bid = 1773
basis, param = HDB.loadBasis("base_%d.pkl"%bid)
trainencoded, trainlabels = HDE.loadEncoded("encoded_%d_train.pkl"%bid)
testencoded, testlabels = HDE.loadEncoded("encoded_%d_train.pkl"%bid)

In [7]:
# train data 
def train(hdc, traindata, trainlabels, testdata, testlabels, param = Config.config):
    train_acc = []
    test_acc = []
    for _ in tqdm_notebook(range(param["epochs"]), desc='epochs'):
        train_acc.append(hdc.fit(traindata, trainlabels, param))
        test_acc.append(hdc.test(testdata, testlabels))
        print("Train: %f \t \t Test: %f"%(train_acc[-1], test_acc[-1]))
        if train_acc[-1] == 1:
            break
    return np.asarray(train_acc), np.asarray(test_acc)

In [8]:
# Should have 90%
hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
train_acc, test_acc = train(hdc, trainencoded, trainlabels, testencoded, testlabels)

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037)] 


Train: 0.146306 	 	 Test: 0.175032
Train: 0.285047 	 	 Test: 0.176963
Train: 0.368099 	 	 Test: 0.167954
Train: 0.422340 	 	 Test: 0.169241
Train: 0.459038 	 	 Test: 0.169884
Train: 0.493481 	 	 Test: 0.166023
Train: 0.522292 	 	 Test: 0.173745
Train: 0.525833 	 	 Test: 0.168597
Train: 0.548527 	 	 Test: 0.164093
Train: 0.573636 	 	 Test: 0.166667
Train: 0.580557 	 	 Test: 0.171171
Train: 0.596652 	 	 Test: 0.162806
Train: 0.600515 	 	 Test: 0.159588
Train: 0.617737 	 	 Test: 0.166667
Train: 0.621600 	 	 Test: 0.165380
Train: 0.644777 	 	 Test: 0.157014
Train: 0.652181 	 	 Test: 0.158945
Train: 0.668759 	 	 Test: 0.154440
Train: 0.672783 	 	 Test: 0.157658
Train: 0.676163 	 	 Test: 0.164093
Train: 0.686464 	 	 Test: 0.162162
Train: 0.688556 	 	 Test: 0.166667
Train: 0.704008 	 	 Test: 0.166667
Train: 0.702076 	 	 Test: 0.165380
Train: 0.711090 	 	 Test: 0.164736
Train: 0.714631 	 	 Test: 0.165380
Train: 0.724288 	 	 Test: 0.162806
Train: 0.727990 	 	 Test: 0.170528
Train: 0.739256 	 	 

(array([0.14630613, 0.28504748, 0.36809915, 0.42234025, 0.4590375 ,
        0.49348141, 0.52229197, 0.52583293, 0.54852728, 0.57363592,
        0.5805569 , 0.59665218, 0.60051505, 0.617737  , 0.62159987,
        0.64477708, 0.65218091, 0.66875905, 0.67278287, 0.67616288,
        0.68646387, 0.68855625, 0.70400773, 0.70207629, 0.71108965,
        0.71463061, 0.72428778, 0.7279897 , 0.7392564 , 0.73941735,
        0.73539353, 0.74987929, 0.76098503, 0.76388218, 0.7563174 ,
        0.76919363, 0.77402221, 0.7873813 , 0.7931756 , 0.79140512,
        0.79108321, 0.80025752, 0.80412039, 0.80991469, 0.81796234,
        0.8132947 , 0.82552712, 0.82713665, 0.8416224 , 0.8413005 ,
        0.8413005 , 0.84725575, 0.86769676, 0.86769676, 0.86174151,
        0.85369387, 0.86592628, 0.86914534, 0.86657009, 0.89151779,
        0.87831965, 0.89747304, 0.88572348, 0.88685015, 0.89441494,
        0.90713021, 0.89602446, 0.89731209, 0.89650732, 0.88379205,
        0.90729116, 0.92676646, 0.91437309, 0.92

In [11]:
hdc.classes

array([[ 1.45059652,  1.85823268,  4.3838977 , ...,  0.68960764,
         3.6772887 , -2.03289408],
       [ 3.98379697, -0.36273483,  3.25583002, ...,  0.41377529,
         2.2391842 ,  2.58729642],
       [-0.61166525, -2.44437767,  1.52885108, ...,  0.4762864 ,
        -3.14459339,  3.03743057],
       ...,
       [-0.77595269,  2.08645182, -0.84871152, ..., -1.17205222,
         0.46915273, -0.54478451],
       [-0.36785616, -1.58836243, -0.14255617, ...,  0.6666144 ,
        -1.47677186, -0.24973132],
       [ 0.44009013, -0.55716497,  0.93944921, ...,  1.31044942,
         0.43441994, -0.11208485]])